In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
##Dataset yolu belirlendi
base_dir = 'C:/Users/ferdi/Desktop/Bitirme Klasörü/Training_Model/tomato' 
train_dir = os.path.join(base_dir, 'train') 

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 10000 images belonging to 10 classes.


In [7]:
## Ebeveyn modelden bilgiler aktarıldı
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [8]:
## Katmanlar donduruldu
base_model.trainable = False

In [9]:
## Modeli optimize edecek katmanlar eklendi
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  #  katmanlar 
predictions = Dense(10, activation='softmax')(x)  # 10 sınıf var


In [10]:
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
## Model derlendi
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
## Eğitim yapıldı
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,epochs=40)

In [ ]:
## Eğitimin grafiği çizildi
acc = history.history['accuracy']
loss = history.history['loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Eğitim Doğruluğu')
plt.legend(loc='lower right')
plt.title('Eğitim Doğruluğu')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Eğitim Kaybı')
plt.legend(loc='upper right')
plt.title('Eğitim Kaybı')
plt.show()


In [14]:
test_dir=os.path.join(base_dir, 'val') 

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,  # Test veri setinin klasör yolu
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 984 images belonging to 10 classes.


In [ ]:
avg_train_accuracy = np.mean(history.history['accuracy'])
avg_train_loss = np.mean(history.history['loss'])
print("Ortalama doğruluk oranı:" , avg_train_accuracy)
print("Ortalama kayıp oranı:", avg_train_loss)

In [ ]:
predictions = model.predict(test_generator, steps=len(test_generator))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

In [19]:
print(len(predicted_classes))

984


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Confusion matrix hesaplama
cm = confusion_matrix(true_classes, predicted_classes, labels=np.arange(10))  # 10 sınıf var

# Confusion matrix çizdirme
fig, ax = plt.subplots(figsize=(10, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.arange(10))
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

# Modelin sınıflandırma raporunu al
report = classification_report(true_classes, predicted_classes, labels=np.arange(10), output_dict=True)

# F1 skorlarını bir DataFrame'e dönüştür
f1_scores = pd.DataFrame(report).transpose()

# F1 skorlarını çıktı olarak göster
print(f1_scores[['f1-score']])

In [22]:
model.save('model_v2.0.0.h5')

In [23]:
model.save('model_v2.0.0.keras')

In [ ]:
keras.saving.save_model(model, 'model_v1.0.1.keras')

In [23]:
img_path='tomato/val/Tomato___Bacterial_spot/0ab54691-ba9f-4c1f-a69b-ec0501df4401___GCREC_Bact.Sp 3170.jpg'

In [ ]:
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)


# Tahmin yap
predictions = model.predict(img_array)
decoded_predictions = decode_predictions(predictions, top=3)[0]  # İlk üç tahmini al

# Tahminleri yazdır
for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
    print(f"{i + 1}: {label} ({score:.2f})")

In [ ]:
## Keras modelini tflite modeline çevirme
import tensorflow as tf

# Keras modelini yeniden yükleme
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/modelV1.keras')

# Modeli TFLite formatına dönüştürme
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# TFLite modelini dosya olarak yeniden kaydetme
model_path = '/content/drive/MyDrive/Colab Notebooks/modelV1.tflite'
with open(model_path, 'wb') as f:
    f.write(tflite_model)